In [ ]:
import cv2
import numpy as np

def detect_copy_move(image_path):
    # Load imag"C:\Users\vaish\OneDrive\Hình ảnh\IMG20240616104840.jpg"
    image = cv2.imread(r'C:\Users\vaish\Downloads\bug.jpg')
    
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Initialize SIFT detector which describe features in image
    sift = cv2.SIFT_create()
    
    # Detect keypoints and descriptors
    keypoints, descriptors = sift.detectAndCompute(blurred, None)
    
    # FLANN based matcher
    #determine the char and behaviour
    
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    matcher = cv2.FlannBasedMatcher(index_params, search_params)
    
    # Match descriptors 
    #numerical representation of image
    matches = matcher.knnMatch(descriptors, descriptors, k=2)
    
    # Ratio test as per Lowe's paper
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    
    # Extract matched keypoints
    src_pts = np.float32([keypoints[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Homography matrix transformation occur in 2 phases
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # Apply mask to get inliers
    matches_mask = mask.ravel().tolist()
    
    # Number of inliers
    inliers = sum(matches_mask)
    
    # Threshold for considering it a copy-move
    threshold = 10
    
    if inliers > threshold:
        print("Copy-move forgery detected!")
    else:
        print("No copy-move forgery detected.")
    
    # Draw matches
    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=None,
                       matchesMask=matches_mask,
                       flags=cv2.DrawMatchesFlags_DEFAULT)
    
    # Draw matches
    img_matches = cv2.drawMatches(image, keypoints, image, keypoints, good_matches, None, **draw_params)
    
    # Show detected regions
    cv2.imshow("Detected Regions", img_matches)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    image_path = (r'C:\Users\vaish\Downloads\bug.jpg')
    detect_copy_move(image_path)


Copy-move forgery detected!
